In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-to-english-translation-sentences/ara_eng.txt


In [2]:
# Install necessary libraries
!pip install transformers datasets sentencepiece scikit-learn
!!pip install bert-score --no-deps
!pip install nltk
!pip install gradio --quiet 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64",

In [3]:
import pandas as pd
import re
import os
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    MarianTokenizer, MarianMTModel, MarianConfig,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, TrainerCallback, GenerationConfig
)
# from nltk.translate.bleu_score import corpus_bleu,SmoothingFunction
from bert_score import score as bert_score



2025-05-12 19:57:19.923658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747079840.361027      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747079840.493434      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:

import string
import unicodedata

# Load the dataset
path = "/kaggle/input/arabic-to-english-translation-sentences/ara_eng.txt"

df = pd.read_csv(path, sep='\t', names=["english", "arabic"], encoding="utf-8")

# Define English text cleaning
def clean_english(text):
    text = text.lower()
    text = unicodedata.normalize("NFKD", text)  # Normalize accented characters
    text = re.sub(r'\([^)]*\)', '', text)  # Remove things in parentheses
    text = re.sub(r'\[.*?\]', '', text)  # Remove things in brackets
    text = re.sub(r"[^a-z0-9.,!?']+", " ", text)  # Keep punctuation that might help context
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Define Arabic text cleaning
def clean_arabic(text):
    text = re.sub(r'[^\u0600-\u06FF\s]', ' ', text)  # Remove non-Arabic characters
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)  # Remove short vowels (tashkeel)
    text = re.sub(r'[\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06ED]', '', text)  # Additional diacritics
    text = re.sub(r'[\d\W_]', ' ', text)  # Remove digits and symbols
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning
df["english"] = df["english"].apply(clean_english)
df["arabic"] = df["arabic"].apply(clean_arabic)

# Filter short or unhelpful sentences
df = df[(df["english"].str.split().str.len() >= 3) & (df["arabic"].str.split().str.len() >= 3)]
df = df[(df["english"].str.len() > 5) & (df["arabic"].str.len() > 5)]

# Remove duplicates
df = df.drop_duplicates(subset=["english"])


In [5]:
# Split into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))


In [6]:
model_name = "Helsinki-NLP/opus-mt-en-ar"

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

In [7]:
# Preprocessing function for tokenizing input/target text
max_length = 128

def preprocess(batch):
    inputs = tokenizer(batch["english"], max_length=max_length, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["arabic"], max_length=max_length, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply preprocessing
train_dataset = train_dataset.map(preprocess, batched=True, remove_columns=["english", "arabic"])
test_dataset = test_dataset.map(preprocess, batched=True, remove_columns=["english", "arabic"])


Map:   0%|          | 0/16875 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4219 [00:00<?, ? examples/s]

In [8]:


def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # إزالة الـ -100 المستخدم في padding
    labels = [[token for token in label if token != -100] for label in labels]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # حساب BLEU
    # references_bleu = [[label.split()] for label in decoded_labels]
    # hypotheses_bleu = [pred.split() for pred in decoded_preds]
    # smoothie = SmoothingFunction().method4
    # bleu = corpus_bleu(references_bleu, hypotheses_bleu, smoothing_function=smoothie)

    # حساب BERTScore
    P, R, F1 = bert_score(decoded_preds, decoded_labels, lang="ar")
    bert_f1 = F1.mean().item()

    return {
        # "bleu": bleu,
        "bertscore_f1": bert_f1
    }


In [9]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    save_total_limit=1,
    predict_with_generate=True,
     report_to="none",

)

class MetricsCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        small_test_df = test_df.sample(n=100, random_state=42)
        english_texts = small_test_df["english"].tolist()
        references = small_test_df["arabic"].tolist()

        preds = generate_translation(english_texts)

     #    # BLEU
     #  bleu_refs = [[ref.split()] for ref in test_df["arabic"].tolist()]
     # bleu_preds = [pred.split() for pred in all_preds]
     # smoothie = SmoothingFunction().method4
     # bleu = corpus_bleu(bleu_refs, bleu_preds, smoothing_function=smoothie)
        # BERTScore
        P, R, F1 = bert_score(preds, references, lang="ar")
        avg_f1 = F1.mean().item()

        print(f"\n✅ Epoch {int(state.epoch)} | BERTScore (F1): {avg_f1:.4f}")
        # print(f"\n✅ Epoch {int(state.epoch)} - BLEU: {bleu:.4f} | BERTScore (F1): {avg_f1:.4f}")

# Define trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    callbacks=[MetricsCallback()]
)


/tmp/ipykernel_31/765604626.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [10]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [11]:
# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,0.756300
200,0.538000
300,0.509200
400,0.496500
500,0.488500
600,0.437500
700,0.427100
800,0.428600
900,0.399100
1000,0.416700


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead

TrainOutput(global_step=5280, training_loss=0.3162766218185425, metrics={'train_runtime': 2586.0641, 'train_samples_per_second': 65.254, 'train_steps_per_second': 2.042, 'total_flos': 5720349081600000.0, 'train_loss': 0.3162766218185425, 'epoch': 10.0})

In [13]:
# حفظ النموذج
model.save_pretrained("saved_model/")

# حفظ المحول
tokenizer.save_pretrained("saved_model/")



('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.json',
 'saved_model/source.spm',
 'saved_model/target.spm',
 'saved_model/added_tokens.json')

In [14]:
import os

for file in os.listdir("saved_model"):
    print(file)


config.json
source.spm
generation_config.json
special_tokens_map.json
vocab.json
tokenizer_config.json
target.spm
model.safetensors


In [15]:

# Generation configuration
generation_config = GenerationConfig(
    max_length=128,
    num_beams=8,
    repetition_penalty=1.2,
    length_penalty=1.0,
    early_stopping=True,
    forced_eos_token_id=tokenizer.eos_token_id,
     bad_words_ids=[[62801]] 
)


def generate_translation(texts, batch_size=4):
    model.eval()
    translations = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
        input_ids = inputs.input_ids.to(model.device)
        attention_mask = inputs.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                generation_config=generation_config  # استخدم الإعدادات هنا
            )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        translations.extend(decoded)

    return translations




In [17]:
import torch
# ✅ ترجمة مجموعة البيانات كاملة
english_texts = test_df["english"].tolist()
all_preds = generate_translation(english_texts)

# التقييم الكامل
preds = all_preds
references = test_df["arabic"].tolist()
hypotheses = preds

P, R, F1 = bert_score(hypotheses, references, lang="ar")
avg_f1 = F1.mean().item()



# bleu_score_value = corpus_bleu([[ref.split()] for ref in references], [pred.split() for pred in hypotheses])

# print(f"\nBLEU Score on Test Set: {bleu_score_value:.4f}")
print(f"BERTScore (F1) on Test Set: {avg_f1:.4f}")

for i in range(2):
    print(f"\n🔹 English: {test_df['english'].iloc[i]}")
    print(f"🔸 Predicted Arabic: {preds[i]}")
    print(f"✅ Actual Arabic: {test_df['arabic'].iloc[i]}")



tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore (F1) on Test Set: 0.8564

🔹 English: this theory consists of three parts.
🔸 Predicted Arabic: تتكون هذه النظرية من ثلاثة أجزاء
✅ Actual Arabic: تتألف هذه النظرية من ثلاثة أجزاء

🔹 English: everything will change.
🔸 Predicted Arabic: كل شيء سيتغير
✅ Actual Arabic: كل شيئ سيتغير


In [18]:
from transformers import MarianMTModel, MarianTokenizer, GenerationConfig
import gradio as gr

# تحميل النموذج المحفوظ
model_path = "saved_model"
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

# إعدادات التوليد
generation_config = GenerationConfig(
    max_length=128,
    num_beams=4,
    length_penalty=1.0,
    early_stopping=True
)

# دالة الترجمة
def generate_translation(texts, batch_size=4):
    model.eval()
    translations = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
        input_ids = inputs.input_ids.to(model.device)
        attention_mask = inputs.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                generation_config=generation_config
            )
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        translations.extend(decoded)
    return translations

# دالة تستخدم في الواجهة
def translate_and_evaluate(input_text):
    translated = generate_translation([input_text])[0]
    return translated

# Gradio Interface
interface = gr.Interface(
    fn=translate_and_evaluate,
    inputs=gr.Textbox(lines=3, placeholder="اكتب الجملة الإنجليزية الموجودة في مجموعة الاختبار..."),
    outputs=gr.Textbox(label="🔸 Predict Translation"),
    title="🔤 Translation English ➜ Arabic",
    description="Enter an English sentence to get its Arabic translation."
)

interface.launch()


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://f7275fded4ebff8b7c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
